# Part 2. Data Cleaning


### Load the unstructured data into a RDD

In [ ]:
import requests
from pyspark.sql import SparkSession

# Best spark configuration for the given data that we tested
spark = (
    SparkSession.builder
    .appName("data_cleaning")
    .config("spark.master", "yarn")
    .config("spark.driver.memory", "2g")
    .config("spark.yarn.am.memory", "2g")
    .config("spark.executor.memory", "4g")
    .config("spark.executor.cores", "4")
    .config("spark.executor.instances", "3")
    .config("spark.task.cpus", "1")
    .config("spark.dynamicAllocation.enabled", "false")
    .getOrCreate()
)

loaded_rdd = spark.sparkContext.textFile("hdfs:///project/raw_temperature_data")

# Printing the first 5 entries
for i in loaded_rdd.take(5):
    print(i)

SN19710:0 2000-01-01T06:00:00.000Z air_temperature:-6.3degC height_above_ground:2m PT0H PT6H 0 C 2 2
SN19710:0 2000-01-01T06:00:00.000Z air_temperature:-6.3degC height_above_ground:2m PT0H PT6H 0 C 2 2
SN19710:0 2000-01-01T06:00:00.000Z air_temperature:-6.3degC height_above_ground:2m PT0H PT6H 0 C 2 2
SN19710:0 2000-01-01T06:00:00.000Z air_temperature:-6.3degC height_above_ground:2m PT0H PT6H 0 C 2 2
SN19710:0 2000-01-01T06:00:00.000Z air_temperature:-6.3degC height_above_ground:2m PT0H PT6H 0 C 2 2


### Process the raw data line by line and convert it into a structured format containing date as the key and temperature and since cosine tranformation as values. Since the data is hourly we convert the datetime to date

In [2]:
from datetime import datetime
import math
import time

# Defining a function to process each line of the log file and extract date as a key and temperature and day of the year as values
def process_line(line):
    fields = line.split(" ")
    
    source_id, ref_time, temperature, height, time_offset, time_res, time_SeriesId,  performanceCategory,  exposureCategory,  qualityCode = [f.strip() for f in fields]
    
    # date as format: 2024-10-01T00:00:00.000Z
    date = ref_time.split("T")[0]
    temp_value = float(temperature.split(":")[1].rstrip("degC"))

    # Converting to datetime object
    date_obj = datetime.strptime(date, "%Y-%m-%d")

    # Finding the day of the year
    day_of_year = date_obj.timetuple().tm_yday

    # Applying sine/cosine transformation to day of the year to capture seasonality
    day_of_year_sin = round(math.sin(2 * math.pi * day_of_year / 365), 2)
    day_of_year_cos = round(math.cos(2 * math.pi * day_of_year / 365), 2)
    
    return (date, (temp_value, day_of_year_sin, day_of_year_cos))


start_time = time.time()

processed_rdd = loaded_rdd.map(process_line)

end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time}")

# Printing the first 10 entries
for i in processed_rdd.take(10):
    print(i)

Execution time: 0.0025539398193359375


('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))
('2000-01-01', (-6.3, 0.02, 1.0))


### Since the data is in hourly format, and we above converted it to daily format, we need to aggregate the data to get the average temperature for each day.

Tested with two different algorithms using groupbyKey and reduceByKey. The reduceByKey is faster and more efficient so we will use that.

In [3]:
import statistics

# Algo 1
# Grouping the temperatures by date and calculating the daily average temperature
# daily_avg_rdd = processed_rdd.groupByKey().mapValues(lambda temps: list(temps)) \
#     .mapValues(lambda temps: (round(statistics.mean([t[0] for t in temps])), temps[0][1], temps[0][2]))

# Algo 2
daily_avg_rdd = processed_rdd \
    .map(lambda t: (t[0], (t[1][0], t[1][1], t[1][2], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1], a[2], a[3] + b[3])) \
    .mapValues(lambda v: (round(v[0] / v[3]), v[1], v[2]))

# Printing the first 10 entries
for i in daily_avg_rdd.take(10):
    print(i)

('2013-01-17', (-8, 0.29, 0.96))
('2013-01-25', (-13, 0.42, 0.91))
('2013-02-10', (-7, 0.65, 0.76))
('2014-05-09', (6, 0.8, -0.61))
('2014-06-14', (15, 0.3, -0.95))
('2014-06-16', (15, 0.26, -0.96))
('2014-08-04', (19, -0.55, -0.84))
('2014-08-27', (16, -0.83, -0.56))
('2014-09-13', (14, -0.95, -0.3))
('2014-09-16', (13, -0.97, -0.25))


### Convert cleaned data to a data frame and save it as a parquet file

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
import pyspark.sql.functions as F

start_time = time.time()

formatted_rdd = daily_avg_rdd.map(lambda x: (x[0], x[1][0], x[1][1], x[1][2]))

# Define the schema for your data
schema = StructType([
    StructField("date", StringType(), True),
    StructField("temperature", IntegerType(), True),
    StructField("feature1", FloatType(), True),
    StructField("feature2", FloatType(), True)
])

## Convert the RDD to a DataFrame
df = spark.createDataFrame(formatted_rdd, schema)

# Order the DataFrame by date so the data is in chronological order
df = df.sort(F.col("date"))


df.write.mode("overwrite").parquet("hdfs:///project/cleaned_data_parquet")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

df.show()
spark.stop()


Execution time: 6.480842113494873 seconds
+----------+-----------+--------+--------+
|      date|temperature|feature1|feature2|
+----------+-----------+--------+--------+
|2000-01-01|         -6|    0.02|     1.0|
|2000-01-02|         -1|    0.03|     1.0|
|2000-01-03|          2|    0.05|     1.0|
|2000-01-04|          2|    0.07|     1.0|
|2000-01-05|         -1|    0.09|     1.0|
|2000-01-06|          5|     0.1|    0.99|
|2000-01-07|          4|    0.12|    0.99|
|2000-01-08|          6|    0.14|    0.99|
|2000-01-09|          1|    0.15|    0.99|
|2000-01-10|          0|    0.17|    0.99|
|2000-01-11|          3|    0.19|    0.98|
|2000-01-12|          2|    0.21|    0.98|
|2000-01-13|          1|    0.22|    0.98|
|2000-01-14|         -2|    0.24|    0.97|
|2000-01-15|         -3|    0.26|    0.97|
|2000-01-16|          0|    0.27|    0.96|
|2000-01-17|          8|    0.29|    0.96|
|2000-01-18|          3|     0.3|    0.95|
|2000-01-19|          1|    0.32|    0.95|
|2000-01-20|

### Find the size of the cleaned data

In [5]:
#Size of the cleaned data
import os
import sys

# Get the size of the cleaned data
size = os.popen("hdfs dfs -du -s -h /project/cleaned_data_parquet").read()
print(size)

45.1 K  135.2 K  /project/cleaned_data_parquet

